(datawrangling)=

# Data Wrangling

>*The garden of life never seems to confine itself to the plots philosophers have
laid out for its convenience. Maybe a few more tractors would do the trick.*
>
>--Roger Zelazny [^note1]

[^note1]: The quote comes from *Home is the Hangman*, published in 1975.]


This is a somewhat strange chapter, even by my standards. My goal in this chapter is to talk a bit more honestly about the realities of working with data than you'll see anywhere else in the book. The problem with real world data sets is that they are *messy*. Very often the data file that you start out with doesn't have the variables stored in the right format for the analysis you want to do. Sometimes might be a lot of missing values in your data set. Sometimes you only want to analyse a subset of the data. Et cetera. In other words, there's a lot of **_data manipulation_** that you need to do, just to get all your data set into the format that you need it. The purpose of this chapter is to provide a basic introduction to all these pragmatic topics. Although the chapter is motivated by the kinds of practical issues that arise when manipulating real data, I'll stick with the practice that I've adopted through most of the book and rely on very small, toy data sets that illustrate the underlying issue. Because this chapter is essentially a collection of "tricks" and doesn't tell a single coherent story, it may be useful to start with a list of topics:

- [Tabulating data](freqtables)
- [Transforming or recoding a variable](transform)
- Section \@ref(mathfunc). Some useful mathematical functions.
- Section \@ref(subset). Extracting a subset of a vector.
- Section \@ref(subsetdataframe). Extracting a subset of a data frame.
- Section \@ref(sort). Sorting, flipping or merging data sets.
- Section \@ref(reshape). Reshaping a data frame.
- Section \@ref(textprocessing). Manipulating text.
- Section \@ref(importing). Opening data from different file types.
- Section \@ref(coercion). Coercing data from one type to another.
- Section \@ref(datastructures). Other important data types.
- Section \@ref(miscdatahandling). Miscellaneous topics.

As you can see, the list of topics that the chapter covers is pretty broad, and there's a *lot* of content there. Even though this is one of the longest and hardest chapters in the book, I'm really only scratching the surface of several fairly different and important topics. My advice, as usual, is to read through the chapter once and try to follow as much of it as you can. Don't worry too much if you can't grasp it all at once, especially the later sections. The rest of the book is only lightly reliant on this chapter, so you can get away with just understanding the basics. However, what you'll probably find is that later on you'll need to flick back to this chapter in order to understand some of the concepts that I refer to here.

(freqtables)=

## Tabulating and cross-tabulating data

A very common task when analysing data is the construction of frequency tables, or cross-tabulation of one variable against another. There are several functions that you can use in Python for that purpose.

Let's start with a simple example. As the father of a small child, I naturally spend a lot of time watching TV shows like *In the Night Garden*, and I have transcribed a short section of the dialogue. Let's make a `pandas` dataframe with two variables, `speaker` and `utterance`. When we take a look at the data, it becomes very clear what happened to my sanity. 

In [9]:
import pandas as pd

data = {'speaker':["upsy-daisy",  "upsy-daisy",  "upsy-daisy",  "upsy-daisy",  "tombliboo",   "tombliboo",   "makka-pakka", "makka-pakka",
  "makka-pakka", "makka-pakka"],
       'utterance':["pip", "pip", "onk", "onk", "ee",  "oo",  "pip", "pip", "onk", "onk"]}

df = pd.DataFrame(data, columns=['speaker','utterance'])

df


,speaker,utterance
0,upsy-daisy,pip
1,upsy-daisy,pip
2,upsy-daisy,onk
3,upsy-daisy,onk
4,tombliboo,ee
5,tombliboo,oo
6,makka-pakka,pip
7,makka-pakka,pip
8,makka-pakka,onk
9,makka-pakka,onk


With these as my data, one task I might find myself needing to do is construct a frequency count of the number of utterances each character produces during the show. As usual, there are more than one way to achieve this, but the `crosstab` method from `pandas` provides an easy way to do this:

In [16]:
pd.crosstab(index = df["speaker"], columns = "count")

col_0,count
speaker,
makka-pakka,4
tombliboo,2
upsy-daisy,4


The output here tells us on the first line that what we’re looking at is a tabulation of the speaker variable. On the second line it lists all the different speakers that exist in the data, and on the third line it tells you how many times that speaker appears in the data. In other words, it’s a frequency table. Notice that we set the argument `columns` to "count". If instead we want to cross-tabulate the speakers with the utterances, we can set `columns` to the "utterances" column in the dataframe:

In [18]:
pd.crosstab(index=df["speaker"], columns=df["utterance"],margins=True)

utterance,ee,onk,oo,pip,All
speaker,,,,,
makka-pakka,0,2,0,2,4
tombliboo,1,0,1,0,2
upsy-daisy,0,2,0,2,4
All,1,4,1,4,10


### Converting a table of counts to a table of proportions

The tabulation commands discussed so far all construct a table of raw frequencies: that is, a count of the total number of cases that satisfy certain conditions. However, often you want your data to be organised in terms of proportions rather than counts. This could be as a proportion of the row totals or the column totals. Currently, these are both just called "All", so let's first save the output of our crosstab to a variable, and rename the row and column totals to "rowtotals" and "coltotals".

In [49]:
tabs = pd.crosstab(index=df["speaker"], columns=df["utterance"],margins=True)

tabs.columns = list(tabs.columns)[0:-1] + ['rowtotals']
tabs.index = list(tabs.index)[0:-1] + ['coltotals']

tabs

,ee,onk,oo,pip,rowtotals
makka-pakka,0,2,0,2,4
tombliboo,1,0,1,0,2
upsy-daisy,0,2,0,2,4
coltotals,1,4,1,4,10


Now we can divide the entire frequency table by the totals in each column:

In [62]:
tabs/tabs.loc['coltotals']

,ee,onk,oo,pip,rowtotals
makka-pakka,0.0,0.5,0.0,0.5,0.4
tombliboo,1.0,0.0,1.0,0.0,0.2
upsy-daisy,0.0,0.5,0.0,0.5,0.4
coltotals,1.0,1.0,1.0,1.0,1.0


The columns sum to one, so we can see that makka-pakka and upsy-daisy each produced 40% of the utterances, while tombliboo only produced 20%. We can also see the proportion of characters associated with each utterance. For instance, whenever the utterance “ee” is made (in this data set), 100% of the time it’s a Tombliboo saying it. 

The procedure to obtain the row-wise proportion, the procedure is slightly different:

In [66]:
tabs.div(tabs["rowtotals"], axis=0)

,ee,onk,oo,pip,rowtotals
makka-pakka,0.0,0.5,0.0,0.5,1.0
tombliboo,0.5,0.0,0.5,0.0,1.0
upsy-daisy,0.0,0.5,0.0,0.5,1.0
coltotals,0.1,0.4,0.1,0.4,1.0


Each row now sums to one, but that’s not true for each column. What we’re looking at here is the proportions of utterances made by each character. In other words, 50% of Makka-Pakka’s utterances are “pip”, and the other 50% are “onk”.

(transform)=

## Transforming and recoding a variable

